In [162]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [163]:
def preprocess(classes, features):
   # Read the dataset
   data = pd.read_csv('./dataset/dry_bean_dataset.csv')
   # Filter rows based on the 'Class' column
   # Perform linear interpolation for missing values in the 'MinorAxisLength' column
   data[features] = data[features].copy().fillna(data[features].mean())
   # Manually perform Min-Max scaling
   for column in features:
      min_val = data[column].min()
      max_val = data[column].max()
      data[column] = (data[column] - min_val) / (max_val - min_val)

   # Shuffle the data
   # data = data.sample(frac=1, random_state=0).reset_index(drop=True)
   data = shuffle(data, random_state=0)
   data.insert(0, 'bias', 1)
   features.append("bias")
   X = data[features].values
   Y = np.where(data['Class'] == classes[0], -1, np.where(data['Class']==classes[1],0,1))
   x_train, x_test, y_train, y_test = train_test_split(
      X, Y, test_size=0.4, random_state=42)
   return x_train, x_test, y_train, y_test

In [164]:
x_train,x_test,y_train,y_test = preprocess(["BOMBAY","CALI","SIRA"],["Area","Perimeter","MajorAxisLength","MinorAxisLength","roundnes"])

x_train.shape

(90, 6)

In [165]:
def sigmoidFunc(x):
    return (1/(1+np.exp(-x)))
def derivative_sigmoid(x):
    return x* (1-x)

In [166]:
def initialize_weights(hiddenlayers,n_neurons):
   weights = [np.random.rand(n_neurons[i], n_neurons[i+1]) for i in range(hiddenlayers+1)]
   return weights

In [167]:
weights = initialize_weights(2,[6,3,4,3])

weights

[array([[0.56680145, 0.16655523, 0.0113789 ],
        [0.44438497, 0.15915044, 0.17634018],
        [0.88482816, 0.48512104, 0.73908617],
        [0.7195033 , 0.85645275, 0.38403923],
        [0.72560035, 0.61116357, 0.5042702 ],
        [0.82504054, 0.93277321, 0.43000407]]),
 array([[0.64722742, 0.76725809, 0.28281159, 0.22082616],
        [0.70151342, 0.14504412, 0.59031741, 0.67909237],
        [0.5280452 , 0.04392   , 0.92605929, 0.1528439 ]]),
 array([[0.31220752, 0.90038562, 0.78253614],
        [0.85113257, 0.25647233, 0.62327353],
        [0.30601262, 0.10359744, 0.83502163],
        [0.15381805, 0.23422715, 0.82625938]])]

In [168]:
# Calculate neuron activation for an input
def activate_input(weights, inputs):
    activation = 0
    for i in range(len(weights.T)):
        for j in range(len(weights)):
            activation += weights[j] * inputs[i][j]
    activation=sigmoidFunc(activation)
    return activation

In [169]:
def activate_hidden(weights, activatedInput):
    activation = 0
    for i in range(len(weights)):
        for j in range(len(weights)):
            activation += weights[j] * activatedInput[i]
    activation=sigmoidFunc(activation)
    return activation

In [170]:
activations = []
new_input = activate_input(weights[0], x_train)
activations.append(new_input)

for i in range(1, len(weights)):
    new_input = activate_hidden(weights[i], new_input)
    activations.append(new_input)

activations


[array([0.99985958, 0.99917836, 0.9917019 ]),
 array([0.9963631 , 0.94582366, 0.99541744, 0.9588511 ]),
 array([0.99821154, 0.99705285, 0.99999355])]

In [171]:
def backprop_out(y_train,activations):
    for i in y_train:
        output_layer_errors = activations[-1] - i
        delta = output_layer_errors * derivative_sigmoid(output_layer_errors)
    return delta

In [172]:
def backprop_hidden(delta,act):
    print(act)
    for j in range(len(act)-1):
        hidden_layer_error = np.dot(delta[j],act)
        hidden_delta = hidden_layer_error * derivative_sigmoid(hidden_layer_error)
    return hidden_delta

In [173]:
Deltas =[]
delta = backprop_out(y_train,activations)
Deltas.append(delta)
activation =activations[:len(activations)-1]
for i in activation[::-1]:
    delta =backprop_hidden(delta,i)
    Deltas.append(delta)
Deltas

[0.9963631  0.94582366 0.99541744 0.9588511 ]
[0.99985958 0.99917836 0.9917019 ]


[array([1.78206502e-03, 2.92980853e-03, 6.45406084e-06]),
 array([4.13521966e-11, 3.72635139e-11, 4.12737387e-11, 3.82970888e-11]),
 array([1.38817953e-21, 1.38628859e-21, 1.36562012e-21])]

In [174]:
def update_weights_out(weights, deltas, activations, learning_rate):
    new_weight = []
    deltas = deltas[::-1]
    for i in range(len(weights)):
            for j in range(len(weights[i])):
                dotprod = np.dot(deltas[j],activations)
                mult = learning_rate * dotprod
                plus = weights[i][j]+mult
                new_weight.append(plus)
    return new_weight

In [175]:
weights_updated_out=[]
for i in range(len(weights)):
    weights_updated_out.append(update_weights_out(weights[i],Deltas[i],activations[i],0.01))
weights_updated_out

[[array([0.56680152, 0.56680152, 0.56680152]),
  array([0.16658453, 0.16658451, 0.16658429]),
  array([0.01139672, 0.01139671, 0.01139657]),
  array([0.44438504, 0.44438504, 0.44438504]),
  array([0.15917973, 0.15917971, 0.15917949]),
  array([0.176358  , 0.17635799, 0.17635785]),
  array([0.88482822, 0.88482822, 0.88482822]),
  array([0.48515033, 0.48515031, 0.48515009]),
  array([0.73910399, 0.73910398, 0.73910384]),
  array([0.71950337, 0.71950337, 0.71950337]),
  array([0.85648204, 0.85648202, 0.8564818 ]),
  array([0.38405705, 0.38405704, 0.3840569 ]),
  array([0.72560041, 0.72560041, 0.72560041]),
  array([0.61119287, 0.61119285, 0.61119263]),
  array([0.50428802, 0.50428801, 0.50428787]),
  array([0.8250406, 0.8250406, 0.8250406]),
  array([0.9328025 , 0.93280248, 0.93280226]),
  array([0.43002188, 0.43002187, 0.43002174])],
 [array([0.64722742, 0.64722742, 0.64722742, 0.64722742]),
  array([0.76725809, 0.76725809, 0.76725809, 0.76725809]),
  array([0.28281159, 0.28281159, 0.282